In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display
pd.set_option('display.max_columns', None)

## Simple Moving Average (SMA)

In [4]:
def sma(df, i, n=9):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm 
    
    if 'SMA' not in df.columns:
        df['SMA'] = np.nan
        df['SMA_diff'] = np.nan
        
    if i in range(0,(n-1)):
        df.iloc[i, df.columns.get_loc('SMA')] = np.nan
    
    sma = df.iloc[(i-(n-1)):i+1]['Close'].mean()
    df.iloc[i, df.columns.get_loc('SMA')] = sma
    
    diff = sma - df.iloc[i-1]['SMA']
    df.iloc[i, df.columns.get_loc('SMA_diff')] = diff

## Stochastic Oscillator

Stochastic = (C-L_n)/(H_n-L_n)*100   
>C = The Most recent closing price  
>L_n = The lowest price traded of the n previous trading sessions  
>H_n = The highest price traded during the same n period  

In [5]:
def stochastic(df, i, n=14):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm 
    
    
    if 'Stochastic' not in df.columns:
        df['Stochastic'] = np.nan
        df['Stochastic_diff'] = np.nan
        
    if i in range(0, n-1):
        df.iloc[i, df.columns.get_loc('Stochastic')] = np.nan
        return 
        
    C = df.iloc[i]['Close']
    Lowest = min(list(df.iloc[i-(n-1): i+1]['Low']))
    Highest = max(list(df.iloc[i-(n-1): i+1]['High']))
        
    value = ((C - Lowest) / (Highest - Lowest)) * 100
        
    df.iloc[i, df.columns.get_loc('Stochastic')] = value 
    
    diff = value - df.iloc[i-1]['Stochastic']
    df.iloc[i, df.columns.get_loc('Stochastic_diff')] = diff 

## Relative Strength Index(RSI)

>RSI = 100 - (100/RS + 1)  
>RS = Average Gain  / Average Loss  
>Average Gain = Sum of Gains over the past 14 periods / 14  
>Average Loss = Sum of Losses over the past 14 periods / 14  

>>Note: Tradingview.com uses ema to calculate Average Gain and Loss, Values will differ

### rsi variables

In [6]:
def rsi_vars(df,i):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    
    if 'rsi_up_var' not in df.columns:
        df['rsi_up_var'] = np.nan
        
    if 'rsi_down_var' not in df.columns:
        df['rsi_down_var'] = np.nan
        
    if i == 0:
        return
    
    current_close = df.iloc[i]['Close']
    previous_close = df.iloc[i-1]['Close']
        
    if current_close > previous_close:
        df.iloc[i, df.columns.get_loc('rsi_up_var')] = current_close - previous_close
        df.iloc[i, df.columns.get_loc('rsi_down_var')] = 0
        
    else:
        df.iloc[i, df.columns.get_loc('rsi_up_var')] = 0
        df.iloc[i, df.columns.get_loc('rsi_down_var')] = previous_close - current_close

### rsi function

In [7]:
def rsi(df, i, n=14):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm 
    
    rsi_vars(df,i)
    
    if 'RSI' not in df.columns:
        df['RSI'] = np.nan
        df['RSI_diff'] = np.nan
        
    if i > n-1:
        avg_up_move = df['rsi_up_var'].iloc[i-(n-1):i+1].mean()
        avg_down_move = df['rsi_down_var'].iloc[i-(n-1):i+1].mean()
            
        relative_strength = avg_up_move / avg_down_move
        rsi = 100-(100/(1 + relative_strength))
            
        df.iloc[i, df.columns.get_loc('RSI')] = rsi
        
        diff = rsi - df.iloc[i-1]['RSI']
        df.iloc[i, df.columns.get_loc('RSI_diff')] = diff 
    

## Rate Of Change (ROC)

ROC = ((Closing Price p - Closing Price p-n) / Closing Price p-n) * 100

>Closing Price p = Closing price of most recent period  
>Closing Price p-n = Closing price n periods before the most recent period

In [8]:
def roc(df, i, n=9):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm 
    
    if 'ROC' not in df.columns:
        df['ROC'] = np.nan
        df['ROC_diff'] = np.nan
        
    if i in range(0, n):
        return
    
    current_close = df.iloc[i]['Close']
    n_close = df.iloc[i - n]['Close']
        
    roc = ((current_close - n_close)/n_close) * 100
    
    df.iloc[i, df.columns.get_loc('ROC')] = roc
    
    diff = roc - df.iloc[i-1]['ROC']
    df.iloc[i, df.columns.get_loc('ROC_diff')] = diff 

## Average True Range (ATR)

ATR = average TR from current to past n bars  
> TR = Max((High-Low),   ABS(High - Previous Close),   ABS(Low - Previous Close))

### true range

In [9]:
def true_range(df,i):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    
    if 'tr' not in df.columns:
        df['tr'] = np.nan
    
    if i == 0:
        return
    
    current_high = df.iloc[i]['High']
    current_low = df.iloc[i]['Low']
    pre_close = df.iloc[i-1]['Close']
        
    method_1 = abs(current_high - current_low)
    method_2 = abs(current_high - pre_close)
    method_3 = abs(current_low - pre_close)
        
    TR = max(method_1, method_2, method_3)
    
    df.iloc[i, df.columns.get_loc('tr')] = TR

### atr function

In [10]:
def atr(df,i,n=14):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm 
    
    true_range(df,i)
    
    if 'ATR' not in df.columns:
        df['ATR'] = np.nan
        df['ATR_diff'] = np.nan
    
    if i > n-1:
        ATR = df.iloc[i-(n-1) : i+1]['tr'].mean()
        
        df.iloc[i, df.columns.get_loc('ATR')] = ATR
        
        diff = ATR - df.iloc[i-1]['ATR']
        df.iloc[i, df.columns.get_loc('ATR_diff')] = diff 

## Average Directional Index (ADX)

ADX = avg((i) DX bars)  
DX = (abs(positive DMI - negative DMI) / (positive DMI + negative DMI)) * 100

> i = bar count  

> positive DMI = avg(positive DX)  
> negative DMI = avg(negative DX)  

> positive DX = ((if high diff > low diff) and (if high diff is > 0)) = high diff, (else) = 0  
> negative DX = ((if low diff > high diff) and (if low diff is > 0)) = low diff, (else) = 0  

> high diff = current high - previous high  
> low diff = current low - previous low

>>Note: Trading view uses EMA over simple moving averages, values may differ.

### adx variables

In [11]:
def adx_vars(df,i):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index

    if 'pos_dx' not in df.columns:
        df['pos_dx'] = np.nan
    
    if 'neg_dx' not in df.columns:
        df['neg_dx'] = np.nan
        
    if i == 0:
        return
    
    #differencing the highs and lows
    current_high = df.iloc[i]['High']
    previous_high = df.iloc[i-1]['High']
    
    current_low = df.iloc[i]['Low']
    previous_low = df.iloc[i-1]['Low']
    
    high_diff = current_high - previous_high
    low_diff = current_low - previous_low
    
    #calculate positive and negative DX
    if (high_diff > low_diff) and (high_diff > 0):
        
        df.iloc[i, df.columns.get_loc('pos_dx')] = high_diff
        df.iloc[i, df.columns.get_loc('neg_dx')] = 0
        
    elif(low_diff > high_diff) and (low_diff > 0):
      
        df.iloc[i, df.columns.get_loc('pos_dx')] = 0
        df.iloc[i, df.columns.get_loc('neg_dx')] = low_diff
        
    else:
        df.iloc[i, df.columns.get_loc('pos_dx')] = 0
        df.iloc[i, df.columns.get_loc('neg_dx')] = 0

### adx function

In [12]:
def adx(df,i,n=14):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm
    
    adx_vars(df,i)
    
    if 'ATR' not in df.columns:
        atr(df,i,n)
        
    if 'dx' not in df.columns:
        df['dx'] = np.nan
    
    if 'ADX' not in df.columns:
        df['ADX'] = np.nan
        df['ADX_diff'] = np.nan
    
    if i > n-1:
        # calculating smooth pos, neg dx
        avg_pos_dx = df.iloc[i-(n-1):i+1]['pos_dx'].mean()
        avg_neg_dx = df.iloc[i-(n-1):i+1]['neg_dx'].mean()
        
        #calculating pos, neg DMI
        ATR = df.iloc[i]['ATR']
        pos_dmi = (avg_pos_dx / ATR) * 100
        neg_dmi = (avg_neg_dx / ATR) * 100
        
        #calculating direction index (dx)
        dx = (abs(pos_dmi - neg_dmi)/ (pos_dmi + neg_dmi))*100
        df.iloc[i, df.columns.get_loc('dx')] = dx
        
        if i > n*2-2:
            adx = df.iloc[i-13:i+1]['dx'].mean()
            df.iloc[i, df.columns.get_loc('ADX')] = adx 
            
            diff = adx - df.iloc[i-1]['ADX']
            df.iloc[i, df.columns.get_loc('ADX_diff')] = diff 